# Discovering The Best Time To Make a Post On Hacker News

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted

Here are the first few rows of the data set:
![Image](hn.png)

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

`Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?`

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

`Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm`

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?
How do Ask HN posts compare to Show HN posts?

Let's start by importing the libraries we need and reading the data set into a list of lists.


In [1]:
from csv import reader #I'll import other libraries as the need arises
file = open('hacker_news.csv')
file = reader(file)
file = list(file)
hn_header = file[0] #removing the column labels
hn = file[1:]



In [2]:
hn_header

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

In [3]:
hn[:5] #sampling the data

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

## Data Analysis
Now that we've removed the headers from `hn`, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [4]:
ask_posts = [] #to hold the ask hn posts
show_posts = [] # to hold the show hn posts
other_posts = [] # to hold other posts
for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


There are 1,744 Ask HN posts, 1,162 Show HN posts and 17,194 post that begin with other titles.

In [5]:
ask_posts[:5]

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14'],
 ['12210105',
  'Ask HN: Looking for Employee #3 How do I do it?',
  '',
  '1',
  '3',
  'sph130',
  '8/2/2016 14:20'],
 ['10394168',
  'Ask HN: Someone offered to buy my browser extension from me. What now?',
  '',
  '28',
  '17',
  'roykolak',
  '10/15/2015 16:38']]

In [6]:
show_posts[:5]

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11'],
 ['10872799',
  'Show HN: GeoScreenshot  Easily test Geo-IP based web pages',
  'https://www.geoscreenshot.com/',
  '1',
  '9',
  'kpsychwave',
  '1/9/2016 20:45']]

Now we have isolated each group of posts we are interested in, we want to find out which posts generate the most interaction, in this case, comments. To this we have to find the average number of comment each posts gets in the Ask HN and Show HN groups.

In [7]:
#Total comments on Ask HN posts
total_ask_comments = 0
for row in ask_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_ask_comments += num_comments

#Avg no of comments per Ask HN post    
avg_ask_comments = (total_ask_comments / len(ask_posts))
print(avg_ask_comments)
    
    

14.038417431192661


In [8]:
#Total comments on Show HN posts
total_show_comments = 0
for row in show_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_show_comments += num_comments

#Avg no of comments per Show HN post    
avg_show_comments = (total_show_comments / len(show_posts))
print(avg_show_comments)

10.31669535283993


We can see from the results above that Ask HN posts perform better,i.e,generates more comments, than Show HN posts thus we'll continue with the Ask HN posts from here.

Remember we are trying to establish what time(hour) is best to make Hacker News posts. To do this we have to find out how Ask HN posts made at different hours perform(calculated as number of comments made per post per hour). 
This involves:

- Calculating the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculating the average number of comments ask posts receive by hour created.

In [9]:
'''here we create a list of list where each lists contains the time a post was created 
and the number of comments it generated'''
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = row[4]
    num_comments = int(num_comments)
    result = [created_at, num_comments]
    result_list.append(result)
    
result_list

[['8/16/2016 9:55', 6],
 ['11/22/2015 13:43', 29],
 ['5/2/2016 10:14', 1],
 ['8/2/2016 14:20', 3],
 ['10/15/2015 16:38', 17],
 ['9/26/2015 23:23', 1],
 ['4/22/2016 12:24', 4],
 ['11/16/2015 9:22', 1],
 ['2/24/2016 17:57', 1],
 ['6/4/2016 17:17', 2],
 ['9/19/2015 17:04', 7],
 ['9/22/2015 13:16', 1],
 ['6/21/2016 15:45', 1],
 ['1/13/2016 21:17', 4],
 ['10/4/2015 21:27', 4],
 ['1/25/2016 20:27', 2],
 ['10/27/2015 2:47', 3],
 ['1/19/2016 12:01', 1],
 ['3/22/2016 2:05', 22],
 ['9/8/2015 14:04', 2],
 ['8/28/2016 18:06', 2],
 ['7/20/2016 13:44', 7],
 ['9/12/2016 16:52', 7],
 ['2/29/2016 17:52', 3],
 ['4/18/2016 15:28', 6],
 ['12/28/2015 14:38', 2],
 ['4/4/2016 3:34', 1],
 ['1/15/2016 21:47', 3],
 ['11/19/2015 5:33', 29],
 ['12/20/2015 3:59', 2],
 ['10/15/2015 21:34', 20],
 ['2/26/2016 19:20', 3],
 ['8/2/2016 18:00', 3],
 ['2/28/2016 1:24', 33],
 ['1/13/2016 9:12', 5],
 ['5/6/2016 1:14', 4],
 ['6/23/2016 13:59', 7],
 ['4/30/2016 17:21', 11],
 ['10/20/2015 19:21', 1],
 ['10/25/2015 15:09', 9],


In [10]:
'''Here we create to dictionaries to store the amount of comments made per hour
and the amount of posts made per hour'''
counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date_time_string = row[0]
    date_time_string = dt.datetime.strptime(date_time_string, '%m/%d/%Y %H:%M') #parsing timestring
    
    hour = date_time_string.strftime('%H') #formatting time string
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
    
    

In [11]:
'''Here we calculate the average number of comments made per post per hour'''
avg_by_hour = []
for item in comments_by_hour:
    avg_by_hour.append([item, (comments_by_hour[item] / counts_by_hour[item])] )


In [12]:
avg_by_hour #24hr clock

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

The list above containing the number of comments made per post per hour is currently difficult to interpret. To make it a bit easy we need to sort the number of comments made per post per hour in descending order.

In [13]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
swap_avg_by_hour

[[5.5777777777777775, '09'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [16.796296296296298, '16'],
 [7.985294117647059, '23'],
 [9.41095890410959, '12'],
 [11.46, '17'],
 [38.5948275862069, '15'],
 [16.009174311926607, '21'],
 [21.525, '20'],
 [23.810344827586206, '02'],
 [13.20183486238532, '18'],
 [7.796296296296297, '03'],
 [10.08695652173913, '05'],
 [10.8, '19'],
 [11.383333333333333, '01'],
 [6.746478873239437, '22'],
 [10.25, '08'],
 [7.170212765957447, '04'],
 [8.127272727272727, '00'],
 [9.022727272727273, '06'],
 [7.852941176470588, '07'],
 [11.051724137931034, '11']]

In [14]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True) # sorts the list in descending order
sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [15]:
sorted_swap[:5] #Best 5 times to make an Ask HN post on HN

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21']]

In the next few cells we'll attempt to make our findings easy to read and interpret.

In [16]:
print('Top 5 Hours for Ask Posts Comments')
for row in sorted_swap[:5]:
    hour = row[1]
    avg_comment = row[0]
    hour = dt.datetime.strptime(hour, '%H')
    hour = hour.strftime('%H:%M')
    print('{}: {:.2f} average comments per post'.format(hour, avg_comment))
    

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


In [17]:
print('Worst 5 Hours for Ask Posts Comments')
for row in sorted_swap[-6:-1]:
    hour = row[1]
    avg_comment = row[0]
    hour = dt.datetime.strptime(hour, '%H')
    hour = hour.strftime('%H:%M')
    print('{}: {:.2f} average comments per post'.format(hour, avg_comment))

Worst 5 Hours for Ask Posts Comments
23:00: 7.99 average comments per post
07:00: 7.85 average comments per post
03:00: 7.80 average comments per post
04:00: 7.17 average comments per post
22:00: 6.75 average comments per post


Now that our results are properly formatted we have one more problem. **TimeZone** . The Time used in the Data Set as seen in the [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts) is Eastern time US Timezone. I am currently in a GMT+1 Timezone so I will convert the time from Eastern(EST) to GMT+1. 

In [18]:
print('Top 5 Hours for Ask Posts Comments')
for row in sorted_swap[:5]:
    hour = row[1]
    avg_comment = row[0]
    hour = dt.datetime.strptime(hour, '%H')
    my_hour = hour + dt.timedelta(hours=6)#EST is 6 hours behind GMT+1
    us_hour = hour.strftime('%H:%M')
    my_hour = my_hour.strftime('%H:%M')
   
    print('{} EST: / {} GMT+1:  {:.2f} average comments per post'.format(us_hour, my_hour, avg_comment))
    


Top 5 Hours for Ask Posts Comments
15:00 EST: / 21:00 GMT+1:  38.59 average comments per post
02:00 EST: / 08:00 GMT+1:  23.81 average comments per post
20:00 EST: / 02:00 GMT+1:  21.52 average comments per post
16:00 EST: / 22:00 GMT+1:  16.80 average comments per post
21:00 EST: / 03:00 GMT+1:  16.01 average comments per post


In [19]:
print('Worst 5 Hours for Ask Posts Comments')
for row in sorted_swap[-5:]:
    hour = row[1]
    avg_comment = row[0]
    hour = dt.datetime.strptime(hour, '%H')
    my_hour = hour + dt.timedelta(hours=6)
    us_hour = hour.strftime('%H:%M')
    my_hour = my_hour.strftime('%H:%M')
   
    print('{} EST: / {} GMT+1:  {:.2f} average comments per post'.format(us_hour, my_hour, avg_comment))

Worst 5 Hours for Ask Posts Comments
07:00 EST: / 13:00 GMT+1:  7.85 average comments per post
03:00 EST: / 09:00 GMT+1:  7.80 average comments per post
04:00 EST: / 10:00 GMT+1:  7.17 average comments per post
22:00 EST: / 04:00 GMT+1:  6.75 average comments per post
09:00 EST: / 15:00 GMT+1:  5.58 average comments per post


## Findings
- The best time to make a Ask HN post on Hacker news is 23:00 EST: / 05:00 GMT+1.
- The worst time to make a Ask HN post on Hacker news is 09:00 EST: / 15:00 GMT+1.
- Posts made in the latter 12 hours of the day(EST) perform better than posts made in the first 12 hours of the day.
- Ask HN posts are more popular than Show HN posts.

## Further Work
- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.
- Compare your results to the average number of comments and points other posts receive.
